In [93]:
import pandas as pd
from timeit import default_timer as timer

In [117]:
import json

with open("secret-key.json", "r") as file:
    key_json = json.load(file)
    
GEMINI_API_KEY = key_json["GEMINI_API_KEY"]

In [119]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [121]:
# response = model.generate_content("Explain how AI works")
# print(response.text)

#### Data Manipulation

In [48]:
def read_file():
    column_types = {
            'ID': 'object',  
            'Tweet': 'object',
            'Cleaned_Tweet': 'object',  
            'Replies': 'int64', 
            'Retweets': 'int64',
            'Likes': 'int64', 
            'Bookmarks': 'int64', 
            'Views': 'int64', 
            'Username': 'object',
            'Hashtags': 'object',
            'Category': 'object',
        }
    df = pd.read_csv('Tweet_data_cleaning.csv', dtype=column_types, parse_dates=['Timestamp', 'PostDate']) #Timestamp และ PostDate เป็น date time
    
    print("cleaned data:")
    display(df)
    
    return df

In [49]:
def prepare_sentiment_data(df):
    columns_for_sentiment = ["ID", "Tweet", "Cleaned_Tweet", "Category"]
    df = df[columns_for_sentiment]
    print("filter columns:")
    display(df)
    return df

In [54]:
def add_sentiment_columns(df):
    # ตรวจสอบว่าคอลัมน์ "Relevance" และ "Sentiment" มีอยู่หรือยัง ถ้าไม่มีก็สร้าง
    if "Relevance" not in df.columns:
        df["Relevance"] = None  # กำหนดค่าเริ่มต้นเป็น None
    if "Sentiment" not in df.columns:
        df["Sentiment"] = None  # กำหนดค่าเริ่มต้นเป็น None
    print("add sentiment columns:")
    display(df)
    return df

In [89]:
def drop_sentiment_columns(df):
    df = df.drop(columns=["Relevance"])
    df = df.drop(columns=["Sentiment"])
    print("drop columns:")
    display(df)
    return df

preprocessing

In [52]:
def preprocess_data():
    data = read_file()
    sentiment_data = prepare_sentiment_data(data)
    add_colums_df = add_sentiment_columns(sentiment_data)
    return add_colums_df

#### Prompt แบบที่ 1

In [77]:
# ฟังก์ชันสำหรับตรวจสอบความเกี่ยวข้อง (Relevance Analysis)
def check_relevance(tweet, category):
    prompt = f"""
    Analyze the following tweet to determine if it contains information or opinions relevant to the given category.  
    If the tweet is related, respond with "relevant"; if not, respond with "irrelevant." Respond in lowercase.

    Tweet: "{tweet}"
    Category: "{category}"
    """
    try:
        response = model.generate_content(prompt)
        return response.text.strip()  # คืนค่าผลลัพธ์ เช่น "relevant" หรือ "irrelevant"
    except Exception as e:
        print(f"Error checking relevance for tweet: {tweet}. Error: {e}")
        return "Error"

In [37]:
# ฟังก์ชันสำหรับ Sentiment Analysis
def analyze_sentiment(text):
    prompt = f"""
    Analyze the sentiment of the following text and classify it as one of the following:
    [positive/negative/neutral]. Respond with only one label in lowercase.

    Text: "{text}"
    """
    try:
        response = model.generate_content(prompt)
        return response.text.strip()  # คืนค่าผลลัพธ์ เช่น "positive"
    except Exception as e:
        print(f"Error analyzing sentiment for text: {text}. Error: {e}")
        return "Error"

In [94]:
def process_prompt_1(df):
    start = timer()
    # ลูปประมวลผลข้อความในคอลัมน์ "Tweet" และ "Category"
    print("show 'irrelevant' and 'negative' text:")
    for index, row in df.iterrows():
        if pd.isna(row["Relevance"]) or pd.isna(row["Sentiment"]):  # ตรวจสอบว่าคอลัมน์ว่ามีค่าหรือไม่
            
            # 1: ตรวจสอบความเกี่ยวข้อง (Relevance)
            relevance = check_relevance(row["Cleaned_Tweet"], row["Category"])
            df.at[index, "Relevance"] = relevance
            
            # 2: ทำ Sentiment Analysis
            sentiment = analyze_sentiment(row["Tweet"])
            df.at[index, "Sentiment"] = sentiment         
            
            if relevance == "irrelevant":
                print("irrelevant")
                print(row["Cleaned_Tweet"], row["Category"])
            if sentiment == "negative":
                print("negative")
                print(row["Cleaned_Tweet"], row["Category"])
                
    print("result of prompt 1:")
    with pd.option_context('display.max_colwidth', None):
        display(df)
    end = timer()
    print("processing time:")
    print(end - start)
    return df

#### Prompt แบบที่ 2

In [72]:
def analyze_tweet(tweet, category):
    prompt = f"""
Classify the sentiment and relevance of the following tweet regarding the specified category.

Relevance:  Determine if the tweet contains information or opinions relevant to the category. Respond with "relevant" or "irrelevant".

Sentiment: Analyze the overall sentiment expressed in the tweet. Classify as "positive", "negative", or "neutral".

Respond in this format:
Relevance: [relevant/irrelevant]
Sentiment: [positive/negative/neutral]

Tweet: "{tweet}"
Category: "{category}"
    """
    try:
        response = model.generate_content(prompt)
        result_lines = response.text.strip().split("\n")  # แยกข้อความตอบกลับเป็นบรรทัด
        relevance = result_lines[0].split(":")[1].strip()  # แยกค่า Relevance
        sentiment = result_lines[1].split(":")[1].strip()  # แยกค่า Sentiment
        return relevance, sentiment
    except Exception as e:
        print(f"Error analyzing tweet: {tweet}. Error: {e}")
        return "irrelevant", "neutral"  # ค่า default กรณี error

In [95]:
def process_prompt_2(df):
    start = timer()
    # ลูปประมวลผลข้อความในคอลัมน์ "Tweet" และ "Category"
    print("show 'irrelevant' and 'negative' text:")
    for index, row in df.iterrows():
        if pd.isna(row["Relevance"]) or pd.isna(row["Sentiment"]):  # ตรวจสอบว่าข้อมูลยังไม่ได้ประมวลผล
            relevance, sentiment = analyze_tweet(row["Cleaned_Tweet"], row["Category"])
            df.at[index, "Relevance"] = relevance
            
            if relevance == "irrelevant":
                print("irrelevant")
                print(row["Cleaned_Tweet"], row["Category"])
            df.at[index, "Sentiment"] = sentiment
            if sentiment == "negative":
                print("negative")
                print(row["Cleaned_Tweet"], row["Category"])
                
    print("result of prompt 2:")
    with pd.option_context('display.max_colwidth', None):
        display(df)
        
    end = timer()
    print("processing time:")
    print(end - start)
    return df

#### LM pipeline

In [98]:
def run_pipeline():
    
    sentiment_data = preprocess_data()
    
    # สร้างผลลัพธ์แยกตาม prompt
    print("result 1:")
    result_1 = process_prompt_1(sentiment_data.copy())
    print("result 2:")
    result_2 = process_prompt_2(sentiment_data.copy())
    
    return result_1, result_2

In [106]:
prompt_1,prompt_2 = run_pipeline()

cleaned data:


,ID,Tweet,Cleaned_Tweet,Replies,Retweets,Likes,Bookmarks,Views,Username,PostDate,Timestamp,Hashtags,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,NaN,0,1,0,0,3040,luckyseyo,2024-10-30 02:07:25,2024-12-24 04:45:53,"['trueล่ม', 'เน็ตทรู', 'True', 'truevision', '...",เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,0,6,22,3,2371,TCASterApp,2024-09-13 11:03:38,2024-12-06 02:22:13,"['TCASter', 'Mytcas', 'TCAS68', 'DEK68', 'admi...",มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,0,0,1,0,131,thecoverageinfo,2023-09-26 10:54:55,2024-11-29 01:24:19,"['แพทยศาสตร์', 'เกษตรศาสตร์', 'แพทยสภา']",มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,0,0,0,0,1252,TCASterApp,2024-11-13 10:24:33,2024-12-24 04:44:51,"['TCASter', 'Mytcas', 'TCAS67', 'TCAS68', 'DEK...",มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,1,12,31,5,3161,TCASterApp,2024-08-06 02:42:39,2024-12-24 04:43:33,"['DEK68', 'TCASter', 'Mytcas', 'TCAS67', 'TCAS...",มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,0,0,0,0,995,Kom_chad_luek,2024-02-25 09:52:02,2024-12-24 04:43:33,"['สุวัจน์', 'เกษตรศาสตร์', 'kurun3']",มหาวิทยาลัยเกษตรศาสตร์
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,0,0,0,0,672,BUSshine__id,2024-12-18 13:07:54,2024-12-24 04:44:51,"['เกษตรศาสตร์', 'ทีมมก', 'kasetsart', 'ku', 'k...",มหาวิทยาลัยเกษตรศาสตร์
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",0,0,0,0,198,kjn_kt,2024-03-23 02:32:43,2024-12-03 22:27:43,"['เน็ตเอไอเอส', 'nPerf']",เน็ตเอไอเอส
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,0,0,0,0,491,naaegyo_shop,2024-11-13 03:34:34,2024-12-24 04:44:51,"['ทีมมก', 'มก', 'เกษตรบางเขน', 'เกษตรศาสตร์', ...",มหาวิทยาลัยเกษตรศาสตร์


filter columns:


,ID,Tweet,Cleaned_Tweet,Category
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonlin...,NaN,เน็ตทรู
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์...,มหาวิทยาลัยเกษตรศาสตร์
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เ...,มหาวิทยาลัยเกษตรศาสตร์
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทย...,มหาวิทยาลัยเกษตรศาสตร์
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยา...,มหาวิทยาลัยเกษตรศาสตร์
...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ช...,มหาวิทยาลัยเกษตรศาสตร์
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : ...,มหาวิทยาลัยเกษตรศาสตร์
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS M...",เน็ตเอไอเอส
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภ...,มหาวิทยาลัยเกษตรศาสตร์


add sentiment columns:


...

result 1:
show 'irrelevant' and 'negative' text:
irrelevant
nan เน็ตทรู
negative
nan เน็ตทรู
irrelevant
*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK มหาวิทยาลัยเกษตรศาสตร์
negative
1242 เจออิมะลิแก้ปัญหาให้ไม่ได้ไม่ยอมให้คุยกับพนักงาน iservice เจออิมะลิกว่าจะได้คุยกับพนักงานรอคิวเป็นชั่วโมง จะแจ้งปัญหาจะสอบถามอะไรไม่เคยได้เลย ต้องมาทวิตด่าให้คนอื่นเขารู้ถึงบริการแย่ๆก่อนถึงจะได้คุยกับคน สุดท้ายโยนความผิดให้ลูกค้ารับจบตลอด @TrueMoveH เน็ตทรู
negative
14.52 น. วันที่ 18 พ.ย. 66 ไฟไหม้ภายใน ม.เกษตรศาสตร์ จนท.รุดควบคุมเพลิง มหาวิทยาลัยเกษตรศาสตร์
negative
@TrueMoveH สอบถามค่ะ ถ้าจะร้องเรียนเจ้าหน้าที่ทรูที่มาหลอกขายโปรกุเกือบพันสามารถทำได้ช่องทางไหนบ้างคะนอกจาก call center อยากเดินไปถามมันมากว่าหลอกขายมากี่คนแล้ว บอกว่าซื้อโปรนี้แถมโปรอื่นฟรี แต่ความจริงมันเป็นโปรแยกมาแล้วต้องเสียเพิ่ม หรือทางทรูเทรนจนทให้หลอกลูกค้าคะ เน็ตทรู
negative
AIS

,ID,Tweet,Cleaned_Tweet,Category,Relevance,Sentiment
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonline #truemoveH,NaN,เน็ตทรู,irrelevant,negative
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ #TCASter #Mytcas #TCAS68 #DEK68 #admission #portfolio #Quota #DirectAdmission #KU #kasetsart #เกษตรศาสตร์,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน #แพทยศาสตร์ #เกษตรศาสตร์ #แพทยสภา https://thecoverage.info/news/content/5464…,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน https://thecoverage.info/news/content/5464…,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #dek69 #KU #เกษตรศาสตร์ #มหาวิทยาลัย #สอบติดมหาลัย,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,positive
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ #DEK68 หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ . #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #admission #รอบ3 #แอดมิชชัน #เกษตร #เกษตรศาสตร์ #หมาวิทยาลัยเกษตร,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ .,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์ #สุวัจน์ #เกษตรศาสตร์ #kurun3,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl… #เกษตรศาสตร์ #ทีมมก #kasetsart #ku #kuopenhouse #tcas #TCAS68 #BUSbecauseofyouishine #THAIchayanon,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl…,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,neutral
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / #เน็ตเอไอเอส / #nPerf v2.13.6 /","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / / v2.13.6 /",เน็ตเอไอเอส,relevant,neutral
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภาวะผู้นำในองค์กร 250฿ ส่งฟรี จากราคาเต็ม 400฿ ☆ #ทีมมก #มก #เกษตรบางเขน #เกษตรศาสตร์ #KU #ส่งต่อหนังสือ #หนังสือมือสอง #หนังสือมือสองสภาพดี #ส่งต่อหนังสือเรียน,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภาวะผู้นำในองค์กร 250฿ ส่งฟรี จากราคาเต็ม 400฿ ☆,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,neutral


processing time:
309.9627308000054
result 2:
show 'irrelevant' and 'negative' text:
irrelevant
nan เน็ตทรู
irrelevant
*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK มหาวิทยาลัยเกษตรศาสตร์
negative
1242 เจออิมะลิแก้ปัญหาให้ไม่ได้ไม่ยอมให้คุยกับพนักงาน iservice เจออิมะลิกว่าจะได้คุยกับพนักงานรอคิวเป็นชั่วโมง จะแจ้งปัญหาจะสอบถามอะไรไม่เคยได้เลย ต้องมาทวิตด่าให้คนอื่นเขารู้ถึงบริการแย่ๆก่อนถึงจะได้คุยกับคน สุดท้ายโยนความผิดให้ลูกค้ารับจบตลอด @TrueMoveH เน็ตทรู
negative
14.52 น. วันที่ 18 พ.ย. 66 ไฟไหม้ภายใน ม.เกษตรศาสตร์ จนท.รุดควบคุมเพลิง มหาวิทยาลัยเกษตรศาสตร์
negative
@TrueMoveH สอบถามค่ะ ถ้าจะร้องเรียนเจ้าหน้าที่ทรูที่มาหลอกขายโปรกุเกือบพันสามารถทำได้ช่องทางไหนบ้างคะนอกจาก call center อยากเดินไปถามมันมากว่าหลอกขายมากี่คนแล้ว บอกว่าซื้อโปรนี้แถมโปรอื่นฟรี แต่ความจริงมันเป็นโปรแยกมาแล้วต้องเสียเพิ่ม หรือทางทรูเทรนจนทให้หลอกลูกค้าคะ เน็ตทร

,ID,Tweet,Cleaned_Tweet,Category,Relevance,Sentiment
0,1851445774359654773,#trueล่ม #เน็ตทรู #True #truevision #trueonline #truemoveH,NaN,เน็ตทรู,irrelevant,neutral
1,1834548488585236847,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ #TCASter #Mytcas #TCAS68 #DEK68 #admission #portfolio #Quota #DirectAdmission #KU #kasetsart #เกษตรศาสตร์,'เปิดครบทั้ง 4 รอบ' แบบไม่มีกั๊ก ม.เกษตรศาสตร์ วิทยาเขตศรีราชา เริ่มรอบพอร์ต 1 ตุลาคมนี้แล้ว! . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
2,1706623376301896183,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน #แพทยศาสตร์ #เกษตรศาสตร์ #แพทยสภา https://thecoverage.info/news/content/5464…,'แพทยสภา' รับรองแล้ว! หลักสูตร 'แพทยศาสตร์ ม.เกษตรฯ' พร้อมเปิดสอนปี 67 รองรับได้ 48 คน https://thecoverage.info/news/content/5464…,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
3,1856644312022618165,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #dek69 #KU #เกษตรศาสตร์ #มหาวิทยาลัย #สอบติดมหาลัย,*อย่าลืมตรวจสอบคุณสมบัติกับประกาศจากทางมหาวิทยาลัยอีกครั้งก่อนสมัคร . ไม่พลาดทุกข่าวสาร TCAS ต้องโหลด TCASter App มาใช้กันนะ : iOS https://tcaster.info/3P6Lu8h : Android https://tcaster.info/3PdmxYK,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,neutral
4,1820651672152285291,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ #DEK68 หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ . #TCASter #Mytcas #TCAS67 #TCAS68 #DEK68 #admission #รอบ3 #แอดมิชชัน #เกษตร #เกษตรศาสตร์ #หมาวิทยาลัยเกษตร,10 คณะแข่งขันสูงจาก TCAS67 มาต่อกันที่มหาวิทยาลัยเกษตรศาสตร์กันบ้าง พี่ทีเชื่อว่าเป็นมหาลัยในฝันของน้อง ๆ หลายคนเลย . จะมีคณะที่การแข่งขันสูงปรี๊ดบ้าง มาดูกันเลย~ .,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
...,...,...,...,...,...,...
346,1761690508253184465,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์ #สุวัจน์ #เกษตรศาสตร์ #kurun3,“สุวัจน์” เปิดงาน เดิน-วิ่ง ม.เกษตร KU RUN 3 ชิงถ้วยพระราชทาน หารายได้สร้างโรงพยาบาลคณะแพทย์ศาสตร์ ม.เกษตรศาสตร์,มหาวิทยาลัยเกษตรศาสตร์,relevant,positive
347,1869368995276841099,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl… #เกษตรศาสตร์ #ทีมมก #kasetsart #ku #kuopenhouse #tcas #TCAS68 #BUSbecauseofyouishine #THAIchayanon,₊· ͟͟͞͞➳❥ Thai IG Story Update — 18.12.2024 : thai.busofficial See u tomorrow https://instagram.com/reel/DDtvzr7BA_B/?igsh=MnM2Nmp6ODF5bzdl…,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,neutral
348,1771364425053896829,"↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / #เน็ตเอไอเอส / #nPerf v2.13.6 /","↓37661 kb/s ↑29640 kb/s, ⇄ 0 ms / Mobile:AIS Mobile / / v2.13.6 /",เน็ตเอไอเอส,relevant,neutral
349,1856541134694040023,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภาวะผู้นำในองค์กร 250฿ ส่งฟรี จากราคาเต็ม 400฿ ☆ #ทีมมก #มก #เกษตรบางเขน #เกษตรศาสตร์ #KU #ส่งต่อหนังสือ #หนังสือมือสอง #หนังสือมือสองสภาพดี #ส่งต่อหนังสือเรียน,☆ ส่งต่อหนังสือเรียนวิชา 01132424 Leadership ภาวะผู้นำในองค์กร 250฿ ส่งฟรี จากราคาเต็ม 400฿ ☆,มหาวิทยาลัยเกษตรศาสตร์,irrelevant,neutral


processing time:
166.35852320000413


#### save result

In [109]:
result_column_types = {
    'ID': 'object',  
    'Tweet': 'object',
    'Cleaned_Tweet': 'object',  
    'Category': 'object',
    'Relevance': 'object',
    'Sentiment': 'object',
}

In [114]:
file_path_promt_1 = "./result_form_sentiment/LM/result_promt_1-1.csv"
file_path_promt_2 = "./result_form_sentiment/LM/result_promt_2-2.csv"

In [112]:
def save_result_to_csv(dataframe, file_path, column_types):
    # เซต Data type
    for column, dtype in column_types.items():
        if column in dataframe.columns:
            dataframe[column] = dataframe[column].astype(dtype)
    
    dataframe.to_csv(
        file_path,
        index=False,
        encoding="utf-8",
    )
    print(f"DataFrame saved successfully to {file_path}")

In [115]:
save_result_to_csv(prompt_1, file_path_promt_1, result_column_types)
save_result_to_csv(prompt_2, file_path_promt_2, result_column_types)

DataFrame saved successfully to ./result_form_sentiment/LM/result_promt_1-1.csv
DataFrame saved successfully to ./result_form_sentiment/LM/result_promt_2-2.csv


#### manual check

check Relevance

In [10]:
prompt = f"""
    Check if the following tweet is relevant to the given category.
    Respond with "relevant" if it is related, otherwise respond with "irrelevant".

    Tweet: "ปีนี้ใครยังไม่มีแพลนจะไปไหน ลองแวะมาลอยกระทงกันได้ที่มหาวิทยาลัย  ในธีม “ ย้อนเวลาไปยุค 80-90 กลิ่นอายความเป็นไทยที่งดงาม ตรวจสอบข้อมูลรายละเอียดการจัดงาน วิธีการเดินทาง พร้อมทั้งไฮไลต์ภายในงาน เกษตรศาสตร์"
    Category: "เกษตรศาสตร์"
    """
response = model.generate_content(prompt)
print(response.text.strip())  # คืนค่าผลลัพธ์ เช่น "relevant" หรือ "irrelevant"

relevant


check Sentiment

In [11]:
prompt = f"""
    Analyze the sentiment of the following text and classify it as one of the following:
    "positive", "negative", or "neutral". Respond with only one label.

    Text: "ปีนี้ใครยังไม่มีแพลนจะไปไหน ลองแวะมาลอยกระทงกันได้ที่มหาวิทยาลัย  ในธีม “ ย้อนเวลาไปยุค 80-90 กลิ่นอายความเป็นไทยที่งดงาม ตรวจสอบข้อมูลรายละเอียดการจัดงาน วิธีการเดินทาง พร้อมทั้งไฮไลต์ภายในงาน เกษตรศาสตร์"
    """

response = model.generate_content(prompt)
print(response.text.strip())  # คืนค่าผลลัพธ์ "positive" "negative" "neutral"

positive
